In [73]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from scipy import interpolate
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [78]:
#w_list = sorted(glob("data/water_data/*.csv"))
df = pd.read_csv("data_2012~2021 impute_tototf+inf+wl.csv")
df_2022 = pd.read_csv("data_2022 impute_tototf+inf.csv")

df = df.replace(" ", np.nan)
df_2022 = df_2022.replace(" ", np.nan)

In [79]:
df.describe()

,Unnamed: 0,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
count,264960.000,264253.000000,264960.000000,264253.000000,264253.000000,264960.000000,260046.000000,264960.000000,251765.000000,264960.000000,79487.0,264960.000000,263681.000000,264960.000000,264901.000000
mean,132479.500,25.072592,562.092284,219.630025,24.370585,544.524190,347.262523,329.895862,728.373508,316.476234,0.0,313.057514,603.419963,300.528570,1064.080321
std,76487.508,0.468083,1105.419376,10.730610,10.730225,1107.336479,174.699839,76.363145,897.650172,67.469334,0.0,63.778508,1381.465277,51.321663,1432.056582
min,0.000,0.000000,0.010000,-0.010000,2.150000,9.000000,39.000000,259.700000,243.290000,246.200000,0.0,247.000000,-3522.710000,241.300000,212.040000
25%,66239.750,24.980000,132.500000,212.140000,14.100000,132.000000,193.000000,288.700000,337.640000,278.200000,0.0,276.000000,207.950000,268.300000,460.030000
50%,132479.500,25.102000,256.000000,217.990000,26.010000,175.700000,328.000000,307.700000,461.960000,298.200000,0.0,296.000000,359.190000,287.300000,692.880000
75%,198719.250,25.195000,526.562500,229.900000,31.860000,507.000000,489.000000,343.700000,752.700000,330.200000,0.0,328.000000,713.260000,317.300000,1159.750000
max,264959.000,25.449000,20015.000000,241.850000,244.010000,18615.260000,842.000000,1287.700000,9405.640000,1146.200000,0.0,1080.000000,25289.170000,843.300000,29854.120000


In [80]:
columns = ["fw_1018662", "fw_1018680","fw_1018683", "fw_1019630"]
print(df.shape)
for column in columns:
    print(f"nan count of {column} : {df[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df[str(column)]<=0).sum()}")
    print("-----------")

print()
print(df_2022.shape)
for column in columns:
    print(f"nan count of {column} : {df_2022[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df_2022[str(column)]<=0).sum()}")
    print("-----------")

(264960, 16)
nan count of fw_1018662 : 13195
under zero count of fw_1018662 : 0
-----------
nan count of fw_1018680 : 185473
under zero count of fw_1018680 : 79487
-----------
nan count of fw_1018683 : 1279
under zero count of fw_1018683 : 26602
-----------
nan count of fw_1019630 : 59
under zero count of fw_1019630 : 0
-----------

(11376, 15)
nan count of fw_1018662 : 3185
under zero count of fw_1018662 : 0
-----------
nan count of fw_1018680 : 11376
under zero count of fw_1018680 : 0
-----------
nan count of fw_1018683 : 0
under zero count of fw_1018683 : 1158
-----------
nan count of fw_1019630 : 0
under zero count of fw_1019630 : 0
-----------


In [81]:
df.drop("fw_1018680", axis=1, inplace=True)
df_2022.drop("fw_1018680", axis=1, inplace=True)

In [82]:
df.loc[df["fw_1018683"] <= 0, "fw_1018683"] = np.nan
df_2022.loc[df_2022["fw_1018683"] <= 0, "fw_1018683"] = np.nan

In [83]:
columns = ["fw_1018662","fw_1018683", "fw_1019630"]
print(df.shape)
for column in columns:
    print(f"nan count of {column} : {df[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df[str(column)]<=0).sum()}")
    print("-----------")

print()
print(df_2022.shape)
for column in columns:
    print(f"nan count of {column} : {df_2022[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df_2022[str(column)]<=0).sum()}")
    print("-----------")

(264960, 15)
nan count of fw_1018662 : 13195
under zero count of fw_1018662 : 0
-----------
nan count of fw_1018683 : 27881
under zero count of fw_1018683 : 0
-----------
nan count of fw_1019630 : 59
under zero count of fw_1019630 : 0
-----------

(11376, 14)
nan count of fw_1018662 : 3185
under zero count of fw_1018662 : 0
-----------
nan count of fw_1018683 : 1158
under zero count of fw_1018683 : 0
-----------
nan count of fw_1019630 : 0
under zero count of fw_1019630 : 0
-----------


In [87]:
# 훈련 데이터 수집 : (tototf, inf) : fw_1018662

train_data = []
train_label = []
train_data_append = train_data.append
train_label_append = train_label.append

df = dict(df)
target_column = "fw_1018662"
for j in tqdm(range(len(df[target_column])-1)):
    # 라벨이 nan이면 패스
    if df[target_column][j+1] != df[target_column][j+1]:
        continue

    # 데이터가 nan이면 패스
    if df[target_column][j] != df[target_column][j]:
        continue

    train_data_append([df["tototf"][j], df["inf"][j], df[target_column][j]])
    train_label_append(df[target_column][j+1])

print(np.array(train_data).shape)
print(np.array(train_label).shape)
train_data = np.array(train_data)
train_label = np.array(train_label)

model = XGBRegressor()
model.fit(train_data, train_label)

# dict 리팩토링 완료

for i in tqdm(range(len(df[target_column])-1)):
    # 타겟이 다음 년도 데이터일 때
    if i % 26496 != 0:
        # 추론할 값이 NAN이어야 함
        if df[target_column][i+1] != df["tototf"][i+1]:
            df[target_column][i+1] = model.predict(np.array([df["tototf"][i], df["inf"][i], df[target_column][i]]).reshape(1,-1))

for i in tqdm(range(len(df_2022[target_column])-1)):
    if df_2022[target_column][i+1] != df_2022["tototf"][i+1]:
        df_2022[target_column][i+1] = model.predict(np.array([df_2022["tototf"][i], df_2022["inf"][i], df_2022[target_column][i]]).reshape(1,-1))


100%|██████████| 264959/264959 [00:06<00:00, 41806.97it/s]


(251081, 3)
(251081,)


 53%|█████▎    | 140274/264959 [08:10<07:16, 285.75it/s]


KeyboardInterrupt: 

In [ ]:
columns = ["fw_1018662"]
print(df.shape)
for column in columns:
    print(f"nan count of {column} : {df[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df[str(column)]<=0).sum()}")
    print("-----------")

print()
print(df_2022.shape)
for column in columns:
    print(f"nan count of {column} : {df_2022[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df_2022[str(column)]<=0).sum()}")
    print("-----------")

In [ ]:
# 훈련 데이터 수집 : (tototf, inf) : fw_1018683

train_data = []
train_label = []
train_data_append = train_data.append
train_label_append = train_label.append

df = dict(df)
target_column = "fw_1018683"
for j in tqdm(range(len(df[target_column])-1)):
    # 라벨이 nan이면 패스
    if df[target_column][j+1] != df[target_column][j+1]:
        continue

    # 데이터가 nan이면 패스
    if df[target_column][j] != df[target_column][j]:
        continue

    train_data_append([df["tototf"][j], df["inf"][j], df[target_column][j]])
    train_label_append(df[target_column][j+1])

print(np.array(train_data).shape)
print(np.array(train_label).shape)
train_data = np.array(train_data)
train_label = np.array(train_label)

model = XGBRegressor()
model.fit(train_data, train_label)

# dict 리팩토링 완료

for i in tqdm(range(len(df[target_column])-1)):
    # 타겟이 다음 년도 데이터일 때
    if i % 26496 != 0:
        # 추론할 값이 NAN이어야 함
        if df[target_column][i+1] != df["tototf"][i+1]:
            df[target_column][i+1] = model.predict(np.array([df["tototf"][i], df["inf"][i], df[target_column][i]]).reshape(1,-1))

for i in tqdm(range(len(df_2022[target_column])-1)):
    if df_2022[target_column][i+1] != df_2022["tototf"][i+1]:
        df_2022[target_column][i+1] = model.predict(np.array([df_2022["tototf"][i], df_2022["inf"][i], df_2022[target_column][i]]).reshape(1,-1))


In [ ]:
columns = ["fw_1018683"]
print(df.shape)
for column in columns:
    print(f"nan count of {column} : {df[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df[str(column)]<=0).sum()}")
    print("-----------")

print()
print(df_2022.shape)
for column in columns:
    print(f"nan count of {column} : {df_2022[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df_2022[str(column)]<=0).sum()}")
    print("-----------")

In [ ]:
# 훈련 데이터 수집 : (tototf, inf) : fw_1019630

train_data = []
train_label = []
train_data_append = train_data.append
train_label_append = train_label.append

df = dict(df)
target_column = "fw_1019630"
for j in tqdm(range(len(df[target_column])-1)):
    # 라벨이 nan이면 패스
    if df[target_column][j+1] != df[target_column][j+1]:
        continue

    # 데이터가 nan이면 패스
    if df[target_column][j] != df[target_column][j]:
        continue

    train_data_append([df["tototf"][j], df["inf"][j], df[target_column][j]])
    train_label_append(df[target_column][j+1])

print(np.array(train_data).shape)
print(np.array(train_label).shape)
train_data = np.array(train_data)
train_label = np.array(train_label)

model = XGBRegressor()
model.fit(train_data, train_label)

# dict 리팩토링 완료

for i in tqdm(range(len(df[target_column])-1)):
    # 타겟이 다음 년도 데이터일 때
    if i % 26496 != 0:
        # 추론할 값이 NAN이어야 함
        if df[target_column][i+1] != df["tototf"][i+1]:
            df[target_column][i+1] = model.predict(np.array([df["tototf"][i], df["inf"][i], df[target_column][i]]).reshape(1,-1))

for i in tqdm(range(len(df_2022[target_column])-1)):
    if df_2022[target_column][i+1] != df_2022["tototf"][i+1]:
        df_2022[target_column][i+1] = model.predict(np.array([df_2022["tototf"][i], df_2022["inf"][i], df_2022[target_column][i]]).reshape(1,-1))


In [ ]:
columns = ["fw_1018662", "fw_1018683", "fw_1019630"]
print(df.shape)
for column in columns:
    print(f"nan count of {column} : {df[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df[str(column)]<=0).sum()}")
    print("-----------")

print()
print(df_2022.shape)
for column in columns:
    print(f"nan count of {column} : {df_2022[str(column)].isna().sum()}")
    print(f"under zero count of {column} : {(df_2022[str(column)]<=0).sum()}")
    print("-----------")

In [37]:
df.to_csv("data_2012~2021 impute_tototf+inf+wl+fw.csv")